<a href="https://colab.research.google.com/github/Ke-erah/Cousera_Capstone/blob/master/Hacker_News_stories_from_September_2015_to_September_2016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import re

In [3]:
from google.colab import files
uploaded = files.upload()

Saving HN_posts_year_to_Sep_26_2016.csv to HN_posts_year_to_Sep_26_2016 (3).csv


In [0]:
import io
hn = pd.read_csv(io.BytesIO(uploaded['HN_posts_year_to_Sep_26_2016.csv']))

In [5]:
hn.head()

,id,title,url,num_points,num_comments,author,created_at
0,12579008,You have two days to comment if you want stem ...,http://www.regulations.gov/document?D=FDA-2015...,1,0,altstar,9/26/2016 3:26
1,12579005,SQLAR the SQLite Archiver,https://www.sqlite.org/sqlar/doc/trunk/README.md,1,0,blacksqr,9/26/2016 3:24
2,12578997,What if we just printed a flatscreen televisio...,https://medium.com/vanmoof/our-secrets-out-f21...,1,0,pavel_lishin,9/26/2016 3:19
3,12578989,algorithmic music,http://cacm.acm.org/magazines/2011/7/109891-al...,1,0,poindontcare,9/26/2016 3:16
4,12578979,How the Data Vault Enables the Next-Gen Data W...,https://www.talend.com/blog/2016/05/12/talend-...,1,0,markgainor1,9/26/2016 3:14


In [6]:
titles1 = hn["title"].tolist()
python_mentions = 0
pattern = "[Pp]ython"
for t in titles1:
    if re.search(pattern, t):
        python_mentions +=1
python_mentions

2572

## Counting Matches with Pandas Method

In [7]:
pattern = '[Pp]ython'
titles = hn['title']
python_mentions1 = titles.str.contains(pattern).sum()
python_mentions1

2572

## Using Regex to Select Data

Similar to boolean indexing

In [8]:
ruby_titles = titles[titles.str.contains("[Rr]uby")]
ruby_titles.head()

1291                      Wrote a simple ps clone in ruby
1497     Ruby Poltergeist gem the best way to scrape data
1624    A proposal of new concurrency model for Ruby 3...
1734    Ruby gem to fetch gocd information as rich mod...
1981                  Ruby vs. Python, the Definitive FAQ
Name: title, dtype: object

## Quantifiers

In [9]:
pattern = "e-?mail"
email_bool = titles.str.contains(pattern)
email_bool.head()

0    False
1    False
2    False
3    False
4    False
Name: title, dtype: bool

In [10]:
email_count = email_bool.sum()
email_count

873

In [11]:
email_titles = titles[email_bool]
email_titles.head()

90      This is what happens when you reply to spam email
173                Correct way to validate email adresses
1010    Obama used a pseudonym in emails with Clinton,...
1012    The most broken part of your user experience i...
1029               Visualization of Clinton email scandal
Name: title, dtype: object

## Character Classes

In [0]:
pattern = "\[\w+\]"
tag_bool = titles.str.contains(pattern)
tag_titles = titles[tag_bool]
tag_count = tag_bool.sum()

In [13]:
tag_counts = tag_titles.shape[0]
tag_counts

5871

## Accessing the Matching Text with Capture Groups

In [14]:
pattern = r"\[(\w+)\]"
tag_freq = titles.str.extract(pattern)[0].value_counts()
tag_freq

pdf            3531
video          1524
audio            65
2015             23
Infographic      18
               ... 
CSS               1
Atellier          1
Spoilers          1
LECTURE           1
MAP               1
Name: 0, Length: 445, dtype: int64

In [15]:
titles.head()
 

0    You have two days to comment if you want stem ...
1                           SQLAR  the SQLite Archiver
2    What if we just printed a flatscreen televisio...
3                                    algorithmic music
4    How the Data Vault Enables the Next-Gen Data W...
Name: title, dtype: object

# **Negative Character Classes**

In [0]:
#Return the first 10 story titles that match the provided regular expression
def first_10_matches(pattern):
  all_matches = titles[titles.str.contains(pattern)]
  first_10 = all_matches.head(10)
  return first_10

In [17]:
java_containing_titles_10 = first_10_matches(r"[Jj]ava")
java_containing_titles_10

139         The State of JavaScript: Front-End Frameworks
734     Show HN: New and Painless Couchbase Java SDK W...
948                    When to use a JavaScript framework
980                JavaScript Conditional Comments for IE
1127    JavaScript vs. C++: Creating the same 3D game ...
1205    Show HN: Free JavaScript video course for Desi...
1244                2016 JavaOne Intel Keynote  32mn Talk
1428    Star Wars Intro Scroller in Pure CSS Without J...
1455             Full-Scale JavaScript (A weekly roundup)
1620    TypeScript, Microsofts JavaScript for big appl...
Name: title, dtype: object

In [0]:
#Return the first 10 story titles that match the provided regular expression
def matches(pattern):
  all_matches = titles[titles.str.contains(pattern)]
  return all_matches

In [19]:
all_java_titles = matches(r'[Jj]ava')
all_java_titles

139           The State of JavaScript: Front-End Frameworks
734       Show HN: New and Painless Couchbase Java SDK W...
948                      When to use a JavaScript framework
980                  JavaScript Conditional Comments for IE
1127      JavaScript vs. C++: Creating the same 3D game ...
                                ...                        
292900    Programming Coordinated Behavior in Java (2010...
292991          Leave a good legacy (in PHP and JavaScript)
293019    Show HN: MockTheClock  A tiny JavaScript libra...
293058    The Latest Twist in the Java IDE Wars: Subscri...
293081          How JavaScript closures work under the hood
Name: title, Length: 3868, dtype: object

We can see that there are a number of matches that contain Java as part of the word JavaScript. We want to exclude these titles from matching so we get an accurate count.

One way to do this is by using negative character classes. Negative character classes are character classes that match every character except a character class. 

Using the negative set [^Ss] to exclude instances like JavaScript and Javascript:

In [20]:
pattern = r'[Jj]ava[^Ss]'
all_java = titles[titles.str.contains(pattern)]
all_java

734       Show HN: New and Painless Couchbase Java SDK W...
1244                  2016 JavaOne Intel Keynote  32mn Talk
1803                    RxJava library for Jersey framework
2569      A Demo App of Zhihu Daily Based on MVP and RxJ...
2592         Swift versus Java: the bitset performance test
                                ...                        
292235                               Oracle cuts Java execs
292485                        Oracle sacks Java evangelists
292560    Java 8: Composing functions using compose and ...
292900    Programming Coordinated Behavior in Java (2010...
293058    The Latest Twist in the Java IDE Wars: Subscri...
Name: title, Length: 1018, dtype: object

# Word Boundaries

While the negative set was effective in removing any bad matches that mention JavaScript, it also had the side-effect of removing any titles where Java occurs at the end of the string.

This is because the negative set [^Ss] must match one character. Instances at the end of a string aren't followed by any characters, so there is no match.

A different approach to take in cases like these is to use the word boundary anchor, specified using the syntax \b. A word boundary matches the position between a word character and a non-word character, or a word character and the start/end of a string.

With the word boundary, our pattern matches the Java at the end of the string.

Let's use the word boundary anchor as part of our regular expression to select the titles that mention Java.

In [21]:
pattern = r'\b[Jj]ava\b'
all_javas = titles[titles.str.contains(pattern)]
all_javas

734       Show HN: New and Painless Couchbase Java SDK W...
2592         Swift versus Java: the bitset performance test
2919                              Scala VS Java: fresh view
3584                                      Var comes to Java
3904      Red Hat Links Java to Microsoft's Visual Studi...
                                ...                        
292235                               Oracle cuts Java execs
292485                        Oracle sacks Java evangelists
292560    Java 8: Composing functions using compose and ...
292900    Programming Coordinated Behavior in Java (2010...
293058    The Latest Twist in the Java IDE Wars: Subscri...
Name: title, Length: 1130, dtype: object

#Matching at the Start and End of Strings

So far, we've used regular expressions to match substrings contained anywhere within text. There are often scenarios where we want to specifically match a pattern at the start and end of strings.

On the previous screen, we learned that the word boundary anchor matches the space between a word character and a non-word character. More generally in regular expressions, an anchor matches something that isn't a character, as opposed to character classes which match specific characters.

Other than the word boundary anchor, the other two most common anchors are the beginning anchor and the end anchor, which represent the start and the end of the string, respectively.

Let's use the beginning and end anchors to count how many titles have tags at the start versus the end of the story title in our Hacker News dataset.

In [22]:
pattern1 = r'^\[\w+\]'
beginning_count = titles.str.contains(pattern1).sum()

pattern2 = r'\[\w+\]$'
ending_count = titles.str.contains(pattern2).sum() 
print(beginning_count)
print(ending_count)

304
5384


# **Using Flags to Modify Regex Patterns**

Up until now, we've been using sets like [Pp] to match different capitalizations in our regular expressions. This strategy works well when there is only one character that has capitalization, but becomes cumbersome when we need to cater for multiple instances.

Within the titles, there are many different formatting styles used to represent the word "email." Here is a list of the variations:
email, Email, e Mail, e mail, E-mail, e-mail, eMail, E-Mail, EMAIL, emails, Emails and E-Mails.

To write a regular expression for this, we would need to use a set for all five letters in email, which would make our regular expression very hard to read.

Instead, we can use flags to specify that our regular expression should ignore case.

**Both re.search() and the pandas regular expression methods** accept an optional **flags** argument. This argument accepts one or more flags, which are special variables in the re module that modify the behavior of the regex interpreter.

A list of all available flags is in the documentation, but by far the most common and the most useful is the **re.IGNORECASE** flag, which is also available using the alias **re.I** for convenience.

When you use this flag, all uppercase letters will match their lowercase equivalents and vice versa.

No matter what the capitalization is, our regular expression matches.

We'll finish this mission by writing a regular expression and count the number of times that email is mentioned in story titles. You'll need to use both ignorecase as well as some of the other regex components you've already learned in this mission.

In [23]:
email_tests = pd.Series(['email', 'Email', 'e Mail', 'e mail', 'E-mail',
              'e-mail', 'eMail', 'E-Mail', 'EMAIL', 'emails', 'Emails',
              'E-Mails'])
pattern = r'\be[-\s]?mails?\b'
email_mentions = titles.str.contains(pattern, flags=re.I).sum()
print(email_mentions)

1653


# **Advanced Regular Expressions**

## Capture Groups

In the previous exercise, we counted the number of mentions of "SQL" in the titles of stories. As we learned in the previous mission, to extract those mentions, we need to do two things:

Use the Series.str.extract() method.
Use a regex capture group.
We define a capture group by wrapping the part of our pattern we want to capture in parentheses. If we want to capture the whole pattern, we just wrap the whole pattern in a pair of parentheses.

Let's look at how we can use a capture group to create a frequency table of the different capitalizations of SQL in our dataset.

In [24]:
pattern = r'(sql)'
sql_capitalizations = titles.str.extract(pattern, flags=re.I)
sql_capitalizations_frequency = sql_capitalizations[0].value_counts()
print(sql_capitalizations_frequency)

SQL    1229
sql      74
Sql      24
Name: 0, dtype: int64


We can extend this analysis by looking at titles that have letters immediately before the "SQL," which is a convention often used to denote different variations or flavors of SQL.

In [25]:
pattern = r'(\w+sql)'
sql_flavors = titles.str.extract(pattern, flags=re.I)
sql_flavors_frequency = sql_flavors[0].value_counts()
print(sql_flavors_frequency)

PostgreSQL       352
MySQL            206
NoSQL            106
Postgresql        16
mysql             13
Mysql             12
MemSQL            10
postgresql         7
WebSQL             5
MSSQL              4
TSQL               4
NOSQL              3
MySql              3
nosql              3
SparkSQL           3
NewSQL             2
PubSubSQL          2
Pgsql              1
SimpleSQL          1
ContinuouSQL       1
PostreSQL          1
CloudSQL           1
FirebirdSQL        1
Hgsql              1
PhxSQL             1
html5sql           1
psql               1
SuperiorMySQL      1
PowerUpSQL         1
NoSql              1
mySql              1
NoNoSQL            1
HTSQL              1
tSQL               1
PrefSQL            1
noSQL              1
DeepSQL            1
HoTTSQL            1
pgSQL              1
KnowSQL            1
Psql               1
Name: 0, dtype: int64


Notice how there is some duplication due to varied capitalization in this frequency table.

In this exercise, we're going to extract the mentions of different SQL flavors into a new column and clean those duplicates by making them all lowercase. We'll then analyze the results to look at the average number of comments for each flavor.

In [26]:
#create a new dataframe, hn_sql, including only rows that mention a SQL flavor.
hn_sql = hn[hn['title'].str.contains(r"\w+SQL", flags=re.I)].copy()
hn_sql.head()

,id,title,url,num_points,num_comments,author,created_at
43,12578514,PostgreSQL RDS pg-stat-ramdisk-size new featur...,http://www.3manuek.com/pgstatramdisksize,1,0,3manuek,9/26/2016 1:16
240,12576116,Bidirectional Replication is coming to Postgre...,http://blog.2ndquadrant.com/bdr-is-coming-to-p...,200,38,iamd3vil,9/25/2016 16:54
257,12576002,A fast PostgreSQL client library for Python: 3...,https://github.com/MagicStack/asyncpg,3,1,arjun27,9/25/2016 16:33
452,12573947,4 N00b MySQL Mistakes Every Programmer Makes,http://devops.com/2016/08/11/4-n00b-mysql-mist...,5,0,agsw,9/25/2016 4:22
569,12572611,Postgresql 9.6.0 release schedule (9-29-2016),https://www.postgresql.org/message-id/27572.14...,1,0,phaas,9/24/2016 20:58


In [27]:
#Create a new column called flavor in the hn_sql dataframe, containing extracted mentions of SQL flavors, defined as:
#Any time 'SQL' is preceded by one or more word characters.
#Ignoring all case variation.

pattern = r'(\w+sql)'
hn_sql['flavor'] = hn_sql['title'].str.extract(pattern, flags = re.I)
hn_sql.head(3)

,id,title,url,num_points,num_comments,author,created_at,flavor
43,12578514,PostgreSQL RDS pg-stat-ramdisk-size new featur...,http://www.3manuek.com/pgstatramdisksize,1,0,3manuek,9/26/2016 1:16,PostgreSQL
240,12576116,Bidirectional Replication is coming to Postgre...,http://blog.2ndquadrant.com/bdr-is-coming-to-p...,200,38,iamd3vil,9/25/2016 16:54,PostgreSQL
257,12576002,A fast PostgreSQL client library for Python: 3...,https://github.com/MagicStack/asyncpg,3,1,arjun27,9/25/2016 16:33,PostgreSQL


In [28]:
#Use the Series.str.lower() method to clean the values in the flavor column by converting them to lowercase. 
#Assign the values back to the column in hn_sql.

hn_sql['flavor'] = hn_sql['flavor'].str.lower()
hn_sql.head(3)

,id,title,url,num_points,num_comments,author,created_at,flavor
43,12578514,PostgreSQL RDS pg-stat-ramdisk-size new featur...,http://www.3manuek.com/pgstatramdisksize,1,0,3manuek,9/26/2016 1:16,postgresql
240,12576116,Bidirectional Replication is coming to Postgre...,http://blog.2ndquadrant.com/bdr-is-coming-to-p...,200,38,iamd3vil,9/25/2016 16:54,postgresql
257,12576002,A fast PostgreSQL client library for Python: 3...,https://github.com/MagicStack/asyncpg,3,1,arjun27,9/25/2016 16:33,postgresql


In [29]:
#Use the DataFrame.pivot_table() method to create a pivot table, sql_pivot.

sql_pivot = hn_sql.pivot_table(values= 'num_comments', index = 'flavor')
sql_pivot.head()

,num_comments
flavor,
cloudsql,5.0
continuousql,7.0
deepsql,0.0
firebirdsql,0.0
hgsql,0.0


## Using capture Groups to Extract Data

All of these examples have a number after the word "Python," which indicates a version number.

We can use capture groups to extract the version of Python that is mentioned most often in our dataset by wrapping parentheses around the part of our regular expression which captures the version number.

We'll use a capture group to capture the version number after the word "Python," and then build a frequency table of the different versions.

In [30]:
#Extract python versions frequency in titles and save in a dictionary
pattern = r'[Pp]ython\s([\d.]+)'
py_versions = titles.str.extract(pattern)[0].value_counts()
py_versions_freq = dict(py_versions)
print(py_versions_freq)

{'3': 108, '3.5': 16, '201': 15, '2': 15, '2.7': 12, '3.6': 10, '101': 7, '2.7.11': 5, '3.5.1': 5, '3.6.0': 4, '3.4': 4, '3.5.0': 3, '4': 3, '2.7.': 3, '5': 3, '2.7.12': 1, '8': 1, '2.': 1, '2.7.2': 1, '5.0': 1, '1.5': 1, '1.8.1': 1, '0.6': 1, '2016': 1, '3.5.2': 1, '1.7': 1}


In [31]:
# Counting mentions of the 'C' Language
pattern = r"\b[Cc]\b[^.+]"

c_lang = titles[titles.str.contains(pattern)]
print(len(c_lang))
c_lang.head(20)

1044


252     Show HN: openemacs  a tiny emacs clone, ? 1024...
457            Talking to C Programmers about C++ [video]
517            Booksbyus/scalable-c: Scalable C  The Book
519                                  Python by the C side
906                                       Tiny C Compiler
914     What is the difference between String and stri...
1662                         Lesser-known C coding styles
2215    UniLua -A pure c# implementation of Lua 5.2 fo...
2558    C# functional language extensions and 'Erlang ...
3024        Local Static Variables in Visual Basic and C#
3178    New release of the unit testing framework cmoc...
3634    RoslynLinqRewrite  Compiles C# by rewriting an...
3959    Ask HN: Any distributed-db has redis-like cust...
4161    How to run C programs on the BeagleBone's PRU ...
4175                     Stabilizing C's Quicksort (2014)
4329     Automatic C library binding generator for Golang
4803           Calling a simple C function from D  strcmp
5383    C Prep

# **Using Lookarounds to Control Matches Based on Surrounding Text**

It looks like we're getting close. In our first 10 matches we have one irrelevant result, which is about "Series C," a term used to represent a particular type of startup fundraising.

Additionally, we've run into the same issue as we did in the previous mission — by using a negative set, we may have eliminated any instances where the last character of the title is "C" (the second last line of output matches in spite of the fact that it ends with "C," because it also has "C" earlier in the string).

Neither of these can be avoided using negative sets, which are used to allow multiple matches for a single character. Instead we'll need a new tool: lookarounds.

Lookarounds let us define a character or sequence of characters that either must or must not come before or after our regex match. There are four types of lookarounds.

The contents of a lookaround can include any other regular expression component.

Sometimes programming languages won't implement support for all lookarounds (notably, lookbehinds are not in the official JavaScript specification). As an example, to get full support in the RegExr tool, you'll need to set it to use the PCRE regex engine.

In this exercise, we're going to use lookarounds to refine the regular expression we build on the last screen to capture mentions of the "C" programming language. 

Let's now use lookarounds to exclude the matches we don't want. We want to:

Keep excluding matches that are followed by . or +, but still match cases where "C" falls at the end of the string.
Exclude matches that have the word 'Series' immediately preceding them.

In [32]:
pattern = r'(?<!Series\s)\b[Cc]\b(?![#\.\+])'
c_language = titles[titles.str.contains(pattern, flags=re.I)]
c_mentions = titles.str.contains(pattern,flags=re.I).sum()
print(c_mentions)
c_language.head(20)

965


252     Show HN: openemacs  a tiny emacs clone, ? 1024...
457            Talking to C Programmers about C++ [video]
517            Booksbyus/scalable-c: Scalable C  The Book
519                                  Python by the C side
906                                       Tiny C Compiler
1662                         Lesser-known C coding styles
2469    The oil and gas we have already tapped will ta...
3178    New release of the unit testing framework cmoc...
3959    Ask HN: Any distributed-db has redis-like cust...
4161    How to run C programs on the BeagleBone's PRU ...
4175                     Stabilizing C's Quicksort (2014)
4329     Automatic C library binding generator for Golang
4803           Calling a simple C function from D  strcmp
5383    C Preprocessor Trick for Implementing Similar ...
5684               Tiny C program to limit volume on OS X
6110    C2ats  An utility to generate ATS interface fr...
6117    Realm Objective-C and Swift 1.1.0  Official Sw...
6281    JavaSc

# **Backreferences: Using Capture Groups in a Regex Pattern**

Let's say we wanted to identify strings that had words with double letters, like the "ee" in "feed." Because we don't know ahead of time what letters might be repeated, we need a way to specify a capture group and then to repeat it. We can do this with backreferences.

Whenever we have one or more capture groups, we can refer to them using integers left to right i.e 1,2,...

Within a regular expression, we can use a backslash followed by that integer to refer to the group.

Let's look at how we could write a regex to capture instances of the same two word characters in a row: 
          r'(\w)\1)

NB: the above doesnt match instances where one of the double lettsers is capital e.g. Aa, Oo

Let's use this technique to identify story titles that have repeated words.

In [33]:
pattern = r'\b(\w+)\s\1\b'
repeated_words = titles[titles.str.contains(pattern)]
repeated_words.head(20)

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


2597     The Log Log Prior for the Frequency of Extrate...
2804         AI Is the New New Thing (And Its Taking Over)
9592     A very very very good vacation responder. The ...
10487    The Last Last Summer:  Donald Trump and the Fa...
10722    Grunt 1.0.1 1+ What is the difference and what...
12493    SAs Public Representatives Are All Hypocrites ...
13489    ElixirConf Slides, What Keeps You from Using U...
14059    If you use use document.write, you suck at Jav...
20024     Ask HN: Allowance for for fun post and comments?
21585                             IEX Active Active system
22062    Why Duck Duck Moose Donated Itself to Khan Aca...
22508              His Name Is Meow Meow, and He Hacks DNA
25109    Solve All Your Performance Issues with with So...
25554             Mirror Mirror: File to Cloud in 25 Lines
27625    Show HN: Term Term Revolution  Dance Dance Rev...
27643                              An anti-torpedo torpedo
29016                                  The new new econo

# **Subtstituting RegEx Matches**

When we learned to work with basic string methods, we used the str.replace() method to replace simple substrings. We can achieve the same with regular expressions using the re.sub() function.

When working in pandas, we can use the Series.str.replace() method, which uses nearly identical syntax

Earlier, we discovered that there were multiple different capitalizations for SQL in our dataset. Let's look at how we could make these uniform with the Series.str.replace() method and a regular expression.



Let's use the same technique to make all the different variations of "email" in the dataset uniform.

In [34]:
pattern = r'e-? ?mail'

titles_clean = titles.str.replace(pattern, 'email', flags=re.I)
titles.head(20)

0     You have two days to comment if you want stem ...
1                            SQLAR  the SQLite Archiver
2     What if we just printed a flatscreen televisio...
3                                     algorithmic music
4     How the Data Vault Enables the Next-Gen Data W...
5                         Saving the Hassle of Shopping
6     Macalifa  A new open-source music app for UWP ...
7     GitHub  theweavrs/Macalifa: A music player wri...
8                         Google Allo  first Impression
9         Advanced Multimedia on the Linux Command Line
10    Ask HN: What TLD do you use for local developm...
11                                           Muroc Maru
12              Why companies make their products worse
13                                Tuning AWS SQS Queues
14                                The Promise of GitHub
15                      Joint R&D Has Its Ups and Downs
16    IBM announces next implementation of Apples Sw...
17      Amazons Algorithms Dont Find You the Bes

# **Extracting Domains from URLs**

Here, we'll extract components of URLs from our dataset. As a reminder, most stories on Hacker News contain a link to an external resource.

The task we will be performing first is extracting the different components of the URLs in order to analyze them. On this screen, we'll start by extracting just the domains.

The domain of each URL excludes the protocol (e.g. https://) and the page path (e.g. /Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429).

There are several ways that you could use regular expressions to extract the domain, but we suggest the following technique:

Using a series of characters that will match the protocol.
Inside a capture group, using a set that will match the character classes used in the domain.
Because all of the URLs either end with the domain, or continue with page path which starts with / (a character not found in any domains), we don't need to cater for this part of the URL in our regular expression.
Once you have extracted the domains, you will be building a frequency table so we can determine the most popular domains.

There are several ways that you could use regular expressions to extract the domain, but we suggest the following technique:

Using a series of characters that will match the protocol.
Inside a capture group, using a set that will match the character classes used in the domain.
Because all of the URLs either end with the domain, or continue with page path which starts with / (a character not found in any domains), we don't need to cater for this part of the URL in our regular expression.

Once you have extracted the domains, you will be building a frequency table so we can determine the most popular domains.

In [35]:
pattern = r'https?://([\w\.\-]+)'
domains = hn['url'].str.extract(pattern, flags=re.I)
top_domains = domains[0].value_counts().head(20)
top_domains

medium.com                 15929
github.com                 14419
www.nytimes.com             5985
www.youtube.com             5232
techcrunch.com              4114
www.theguardian.com         3429
arstechnica.com             2979
www.bloomberg.com           2742
en.wikipedia.org            2321
www.bbc.com                 1992
www.wsj.com                 1979
www.washingtonpost.com      1939
www.wired.com               1827
www.theverge.com            1811
www.theatlantic.com         1515
www.bbc.co.uk               1335
qz.com                      1200
www.businessinsider.com     1191
nautil.us                   1189
motherboard.vice.com        1133
Name: 0, dtype: int64

# **Extracting URL Parts Using Multiple Capture Groups**

Having extracted just the domains from the URLs, on this final screen we'll extract each of the three component parts of the URLs:

1. Protocol
2. Domain
3. Page path

In order to do this, we'll create a regular expression with multiple capture groups.

The result is a dataframe with each of our capture groups defining a column of data.

Now let's write a regular expression that will extract the URL components into individual columns of a dataframe.

In [36]:
pattern = r'(https?)://([\w\.\-]+)/?(.*)'
url_parts = hn['url'].str.extract(pattern, flags=re.I)
url_parts.head()

,0,1,2
0,http,www.regulations.gov,document?D=FDA-2015-D-3719-0018
1,https,www.sqlite.org,sqlar/doc/trunk/README.md
2,https,medium.com,vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43
3,http,cacm.acm.org,magazines/2011/7/109891-algorithmic-compositio...
4,https,www.talend.com,blog/2016/05/12/talend-and-Âthe-data-vaultÂ


In [38]:
pattern = r"(?P<protocol>https?)://(?P<domain>[\w\.\-]+)/?(?P<path>.*)"
url_parts=hn['url'].str.extract(pattern, flags=re.I)
url_parts.head(20)

,protocol,domain,path
0,http,www.regulations.gov,document?D=FDA-2015-D-3719-0018
1,https,www.sqlite.org,sqlar/doc/trunk/README.md
2,https,medium.com,vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43
3,http,cacm.acm.org,magazines/2011/7/109891-algorithmic-compositio...
4,https,www.talend.com,blog/2016/05/12/talend-and-Âthe-data-vaultÂ
5,https,blog.menswr.com,2016/09/07/whats-new-with-your-style-feed/
6,http,forums.windowscentral.com,windows-phone-apps/440523-macalifa-new-open-so...
7,https,github.com,theweavrs/Macalifa
8,http,prodissues.com,2016/09/google-allo-first-impression.html
9,https,avi.alkalay.net,2016/09/multimedia-linux-cli.html
